In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load datasets
data_train = pd.read_excel('/content/drive/MyDrive/Thesis - Sentiment Analysis Dataset/1. experiment 2/1. Dataset/training_data.xlsx', sheet_name='Sheet1')
data_test = pd.read_excel('/content/drive/MyDrive/Thesis - Sentiment Analysis Dataset/1. experiment 2/1. Dataset/test_data.xlsx', sheet_name='Sheet1')

# Initialize and fit the label encoder
label_encoder = LabelEncoder()
data_train['label'] = label_encoder.fit_transform(data_train['label'])
data_test['label'] = label_encoder.transform(data_test['label'])

# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

class SentimentDataset(Dataset):
    def __init__(self, tokenizer, texts, labels=None):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokenized = self.tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=128, return_tensors="pt")
        item = {key: val.squeeze() for key, val in tokenized.items()}
        if self.labels is not None:
            item['labels'] = self.labels[idx]
        return item

# Prepare datasets
train_dataset = SentimentDataset(tokenizer, data_train['text'].tolist(), data_train['label'].values)
test_dataset = SentimentDataset(tokenizer, data_test['text'].tolist(), data_test['label'].values)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
model = AutoModelForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1", num_labels=len(label_encoder.classes_))

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# ! pip install -U accelerate
# ! pip install -U transformers

In [4]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


In [5]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=64,  # Adjust based on your GPU memory
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    learning_rate=3e-5,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics  # We will define this function in the next step
)


In [ ]:
trainer.train()
eval_result = trainer.evaluate()


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix

# Predictions
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=-1)

# Confusion Matrix
cm = confusion_matrix(data_test['label'], pred_labels)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()
